In [1]:
# Copyright (c) 2021 The MATCH Authors. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

%reload_ext autoreload
%autoreload 2

from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objects as go 
import numpy as np
from switch_model.reporting.report_functions import *
import pickle

# TODO: remove input folder = '../../MODEL_RUNS/test_PCE/inputs/hourly_90'

cambium_dir = Path.cwd() / '../../MODEL_RUNS/test_excess_penalty_w_hedge/cambium'

#define formatting options/functions for outputs
pd.options.display.float_format = '{:,.2f}'.format

year = 2025

In [2]:
if year % 2 == 0:
    print("number is even")

In [5]:
cambium = pd.read_csv(cambium_dir / 'StdScen20_MidCase_hourly_CA_2024.csv', skiprows=2)
cambium

,timestamp,timestamp_local,busbar_load,busbar_load_for_enduse,enduse_load,storage_charging,net_load_busbar,battery_charging,phs_charging,trans_losses,...,geothermal_MWh,biomass_MWh,wind-ons_MWh,wind-ofs_MWh,csp_MWh,upv_MWh,distpv_MWh,phs_MWh,battery_MWh,canada_MWh
0,2024-01-01 03:00:00,2024-01-01 00:00:00,"26,739.00","26,658.00","25,466.00",0.00,"25,859.20",0.00,0.00,81.00,...,"2,829.20",677.50,798.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2024-01-01 04:00:00,2024-01-01 01:00:00,"25,449.90","25,388.10","24,268.60",0.00,"24,527.30",0.00,0.00,61.80,...,"2,540.00",677.50,860.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2024-01-01 05:00:00,2024-01-01 02:00:00,"24,518.00","24,463.10","23,394.10",0.00,"23,607.20",0.00,0.00,54.90,...,"2,540.00",677.50,856.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2024-01-01 06:00:00,2024-01-01 03:00:00,"23,896.20","23,841.70","22,805.50",0.00,"23,022.70",0.00,0.00,54.50,...,"2,540.00",677.50,819.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2024-01-01 07:00:00,2024-01-01 04:00:00,"23,784.30","23,731.90","22,701.50",0.00,"22,853.50",0.00,0.00,52.40,...,"2,540.00",677.50,878.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2024-12-31 22:00:00,2024-12-31 19:00:00,"35,345.50","35,073.40","33,310.70",0.00,"34,506.50",0.00,0.00,272.20,...,"2,466.00",548.70,566.90,0.00,0.00,0.00,0.00,0.00,827.80,0.00
8756,2024-12-31 23:00:00,2024-12-31 20:00:00,"34,602.40","34,363.40","32,654.90",0.00,"33,805.50",0.00,0.00,239.00,...,"2,755.20",747.60,557.90,0.00,0.00,0.00,0.00,0.00,473.60,0.00
8757,2024-01-01 00:00:00,2024-12-31 21:00:00,"31,998.90","31,809.20","30,285.60",0.00,"31,593.60",0.00,0.00,189.70,...,"2,829.20",679.70,215.50,0.00,0.00,0.00,0.00,649.20,"1,581.30",0.00
8758,2024-01-01 01:00:00,2024-12-31 22:00:00,"30,633.00","30,417.20","28,989.00",0.00,"30,267.90",0.00,0.00,215.70,...,"2,829.20",679.70,149.30,0.00,0.00,0.00,0.00,16.80,"1,045.90",0.00


In [29]:
# calculate system metrics

system_metrics = cambium.copy()[['timestamp_local','net_load_busbar']].set_index('timestamp_local')
system_metrics.index = pd.to_datetime(system_metrics.index)

peak_demand = system_metrics.groupby(system_metrics.index.date).max()
peak_demand.index = pd.to_datetime(peak_demand.index)
peak_demand = peak_demand.groupby(peak_demand.index.quarter).mean()

peak_hour = system_metrics.groupby(system_metrics.index.date).idxmax()
peak_hour['net_load_busbar'] = peak_hour['net_load_busbar'].dt.hour
peak_hour.index = pd.to_datetime(peak_hour.index)
peak_hour = peak_hour.groupby(peak_hour.index.quarter).mean()
#peak_hour = peak_hour.apply(lambda row: f'{int(row["net_load_busbar"])}')
peak_hour


,net_load_busbar
1,18.52
2,19.98
3,18.90
4,18.03
